<a href="https://colab.research.google.com/github/shadowm-mounarch/shadow-commander/blob/main/shadow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers datasets

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

# ===== 0. Detect device =====
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# ===== 1. Load dataset =====
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
train_ds = dataset["train"].select(range(500))
val_ds = dataset["validation"].select(range(100))

# ===== 2. Load tokenizer and model =====
model_name = "EleutherAI/gpt-neo-125M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model.gradient_checkpointing_enable()  # save memory

# ===== 3. Tokenize =====
def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=64)

train_tok = train_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
val_tok = val_ds.map(tokenize_fn, batched=True, remove_columns=["text"])

# ===== 4. Data collator =====
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# ===== 5. Training arguments =====
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=100,
    eval_steps=50,
    eval_strategy="steps",
    save_total_limit=1,
    report_to="none",
    fp16=True if device=="cuda" else False,  # FP16 only on GPU
)

# ===== 6. Trainer =====
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# ===== 7. Train =====
trainer.train()


Exception ignored in: <function _xla_gc_callback at 0x7df9a92cf9c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/jax/_src/lib/__init__.py", line 127, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

wikitext-2-raw-v1/test-00000-of-00001.pa(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

...

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/tmp/ipython-input-3692402282.py:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
50,4.315200,nan
100,2.962600,nan
150,3.600600,nan
200,2.818700,nan
250,3.017200,nan
300,3.288200,nan
350,3.771800,nan
400,2.306800,nan
450,2.768400,nan
500,2.665600,nan


TrainOutput(global_step=500, training_loss=2.9130665893554686, metrics={'train_runtime': 209.5295, 'train_samples_per_second': 2.386, 'train_steps_per_second': 2.386, 'total_flos': 16325443584000.0, 'train_loss': 2.9130665893554686, 'epoch': 1.0})